# Genetic Program Example - adapted from TinyGP by Moshesipper

Import libraries

In [1]:
from random import random, randint, seed, randrange
from statistics import mean
from copy import deepcopy
import numpy as np
from itertools import combinations
import math

## Define parameters

We must now define our parameters, and allow for bloat control.

In [2]:
POP_SIZE        = 60   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 5    # maximal initial random tree depth
GENERATIONS     = 100  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 

## Defining non-terminals and terminals

Define functions for non terminal set - this can be done natively in functional programming languages.

In [3]:
# functions taking two arguments
def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y
def div(x,y): 
    if y != 0:
        return x/y
    else: 
        return 2**(31-1)

In [4]:
# functions taking one argument
def sin(x): return math.sin(x)
def cos(x): return math.cos(x)
def aBs(x): return abs(x)
def eXp(x): 
    return math.exp(x)
def power_1(x): return x
def power_2(x): return x**2
def power_3(x): return x**3
def power_4(x): return x**4
def power_reverse_1(x): return x
def power_reverse_2(x): 
    return x**(1/2)
def power_reverse_3(x): return x**(1/3)
def power_reverse_4(x): return x**(1/4)

Define terminal and non-terminal sets

In [5]:
FUNCTIONS_2_ARGS = [add, sub, mul]
FUNCTIONS_1_ARG = [sin, cos, aBs, eXp, power_1, power_2, power_3, power_4, power_reverse_1, power_reverse_2,
                                                                        power_reverse_3, power_reverse_4]

def generate_terminals(dim):
    TERMINALS = [1, 2, 3, 4, 10, 100, math.pi]
    for i in range(1,dim+1):
        x = f'x{i}'
        TERMINALS.append(x)
    return TERMINALS

TERMINALS = generate_terminals(2)
print(generate_terminals(3))

[1, 2, 3, 4, 10, 100, 3.141592653589793, 'x1', 'x2', 'x3']


## Managing our dataset

In usual settings you will have a dataset which you are working from, much in the same way as in traditional ML tasks; however, for the assignment and for observing, we will consider a target function and a create a dataset from that.

In [6]:
# def target_func(x): # evolution's target
#     return x*x + 2*x + 1

In [7]:
# def generate_dataset(): # generate 101 data points from target_func
#     dataset = []
#     for x in range(-100,101,2): 
#         x /= 100
#         dataset.append([x, target_func(x)])
#     return dataset

## Sphere method

In [8]:
def generate_dataset(dims): # generate 101 data points from target_func
    dataset = []
    f = open('data.txt','r')
    for line in f:
        if len(line) > 1:
            data_split = line.split()
            data = ([float(data_split[0]),float(data_split[1])],float(data_split[2]))
            dataset.append(data)
    return dataset

def flip_coin():
    if randrange(0,100) > 50:
        return True
    else:
        return False

In [9]:
generate_dataset(2)

[([-1.0, -1.0], 0.99372),
 ([-1.0, -0.9], 1.03403),
 ([-1.0, -0.8], 1.17694),
 ([-1.0, -0.7], 1.39572),
 ([-1.0, -0.6], 1.6679),
 ([-1.0, -0.5], 2.02182),
 ([-1.0, -0.4], 2.31388),
 ([-1.0, -0.3], 2.56489),
 ([-1.0, -0.2], 2.81409),
 ([-1.0, -0.1], 2.9631),
 ([-1.0, 0.0], 3.01865),
 ([-1.0, 0.1], 2.93769),
 ([-1.0, 0.2], 2.78573),
 ([-1.0, 0.3], 2.56562),
 ([-1.0, 0.4], 2.30441),
 ([-1.0, 0.5], 1.98337),
 ([-1.0, 0.6], 1.69169),
 ([-1.0, 0.7], 1.40742),
 ([-1.0, 0.8], 1.18464),
 ([-1.0, 0.9], 1.07136),
 ([-0.9, -1.0], 0.322869),
 ([-0.9, -0.9], 0.358391),
 ([-0.9, -0.8], 0.521266),
 ([-0.9, -0.7], 0.711846),
 ([-0.9, -0.6], 0.999958),
 ([-0.9, -0.5], 1.28843),
 ([-0.9, -0.4], 1.62771),
 ([-0.9, -0.3], 1.9024),
 ([-0.9, -0.2], 2.13793),
 ([-0.9, -0.1], 2.26903),
 ([-0.9, 0.0], 2.33338),
 ([-0.9, 0.1], 2.2526),
 ([-0.9, 0.2], 2.10338),
 ([-0.9, 0.3], 1.89028),
 ([-0.9, 0.4], 1.6385),
 ([-0.9, 0.5], 1.30133),
 ([-0.9, 0.6], 0.97772),
 ([-0.9, 0.7], 0.726569),
 ([-0.9, 0.8], 0.515669),
 ([

In [10]:
dataset = generate_dataset(2)
dataset_result = dataset
print(dataset_result)

[([-1.0, -1.0], 0.99372), ([-1.0, -0.9], 1.03403), ([-1.0, -0.8], 1.17694), ([-1.0, -0.7], 1.39572), ([-1.0, -0.6], 1.6679), ([-1.0, -0.5], 2.02182), ([-1.0, -0.4], 2.31388), ([-1.0, -0.3], 2.56489), ([-1.0, -0.2], 2.81409), ([-1.0, -0.1], 2.9631), ([-1.0, 0.0], 3.01865), ([-1.0, 0.1], 2.93769), ([-1.0, 0.2], 2.78573), ([-1.0, 0.3], 2.56562), ([-1.0, 0.4], 2.30441), ([-1.0, 0.5], 1.98337), ([-1.0, 0.6], 1.69169), ([-1.0, 0.7], 1.40742), ([-1.0, 0.8], 1.18464), ([-1.0, 0.9], 1.07136), ([-0.9, -1.0], 0.322869), ([-0.9, -0.9], 0.358391), ([-0.9, -0.8], 0.521266), ([-0.9, -0.7], 0.711846), ([-0.9, -0.6], 0.999958), ([-0.9, -0.5], 1.28843), ([-0.9, -0.4], 1.62771), ([-0.9, -0.3], 1.9024), ([-0.9, -0.2], 2.13793), ([-0.9, -0.1], 2.26903), ([-0.9, 0.0], 2.33338), ([-0.9, 0.1], 2.2526), ([-0.9, 0.2], 2.10338), ([-0.9, 0.3], 1.89028), ([-0.9, 0.4], 1.6385), ([-0.9, 0.5], 1.30133), ([-0.9, 0.6], 0.97772), ([-0.9, 0.7], 0.726569), ([-0.9, 0.8], 0.515669), ([-0.9, 0.9], 0.358813), ([-0.8, -1.0], -

## Creating the genetic program class

In [11]:
class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left
        self.right = right
        self.global_str = ''
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS_2_ARGS or self.data in FUNCTIONS_1_ARG):
            return self.data.__name__
        else: 
            return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")
            
    def generate_str_tree(self):
        self.global_str += self.node_label() + " "
        if self.left:
            self.left.generate_str_tree()
        if self.right:
            self.right.generate_str_tree()
        return self.global_str

    def compute_tree(self, x): 
        if (self.data in FUNCTIONS_2_ARGS): 
            return self.data(self.left.compute_tree(x), self.right.compute_tree(x))
        elif (self.data in FUNCTIONS_1_ARG):
            return self.data(self.left.compute_tree(x))
        elif type(self.data) is str: 
            if self.data[0] == 'x': return x[int(self.data[1:])-1]
        else: return self.data
            
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            if flip_coin() == True:
                self.data = FUNCTIONS_2_ARGS[randint(0, len(FUNCTIONS_2_ARGS)-1)]
            else:
                self.data = FUNCTIONS_1_ARG[randint(0, len(FUNCTIONS_1_ARG)-1)]
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                if flip_coin() == True:
                    self.data = FUNCTIONS_2_ARGS[randint(0, len(FUNCTIONS_2_ARGS)-1)]
                else:
                    self.data = FUNCTIONS_1_ARG[randint(0, len(FUNCTIONS_1_ARG)-1)]
        if self.data in FUNCTIONS_2_ARGS:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)
        elif self.data in FUNCTIONS_1_ARG:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)

    def mutation(self):
        if random() < PROB_MUTATION: # mutate at this node
            self.random_tree(grow = True, max_depth = 2)
        elif self.left: self.left.mutation()
        elif self.right: self.right.mutation() 

    def size(self): # tree size in nodes
        if self.data in TERMINALS: return 1
        l = self.left.size()  if self.left  else 0
        r = self.right.size() if self.right else 0
        return 1 + l + r

    def build_subtree(self): # count is list in order to pass "by reference"
        t = GPTree()
        t.data = self.data
        if self.left:  t.left  = self.left.build_subtree()
        if self.right: t.right = self.right.build_subtree()
        return t
                        
    def scan_tree(self, count, second): # note: count is list, so it's passed "by reference"
        count[0] -= 1            
        if count[0] <= 1: 
            if not second: # return subtree rooted here
                return self.build_subtree()
            else: # glue subtree here
                self.data  = second.data
                self.left  = second.left
                self.right = second.right
        else:  
            ret = None              
            if self.left  and count[0] > 1: ret = self.left.scan_tree(count, second)  
            if self.right and count[0] > 1: ret = self.right.scan_tree(count, second)  
            return ret

    def crossover(self, other): # xo 2 trees at random nodes
        if random() < XO_RATE:
            second = other.scan_tree([randint(1, other.size())], None) # 2nd random subtree
            self.scan_tree([randint(1, self.size())], second) # 2nd subtree "glued" inside 1st tree

## Fitness and selection

In [12]:
def fitness(individual, dataset): # inverse mean absolute error over dataset normalized to [0,1]
    try:
        return 1 / (1 + mean([abs(individual.compute_tree(ds[0]) - ds[1]) for ds in dataset]))
    except Exception:
        return 0
        

In the example we are using tournament based fitness. What benefits and negatives does tournament selection have?

In [13]:
def selection(population, fitnesses): # select one individual using tournament selection
    tournament = [randint(0, len(population)-1) for i in range(TOURNAMENT_SIZE)] # select tournament contenders
    tournament_fitnesses = [fitnesses[tournament[i]] for i in range(TOURNAMENT_SIZE)]
    return deepcopy(population[tournament[tournament_fitnesses.index(max(tournament_fitnesses))]]) 

Try to implement a roulette wheel selection for this and compare your results.

In [14]:
def roulette_selection(population, fitnesses):
    pass

In [15]:
def init_population(): # ramped half-and-half
    pop = []
    for md in range(3, MAX_DEPTH + 1):
        for i in range(int(POP_SIZE/6)):
            t = GPTree()
            t.random_tree(grow = True, max_depth = md) # grow
            pop.append(t) 
        for i in range(int(POP_SIZE/6)):
            t = GPTree()
            t.random_tree(grow = False, max_depth = md) # full
            pop.append(t) 
    return pop

##  Main Loop

In [16]:
def question4():
    dims = 2
    dataset = generate_dataset(dims)
    population = init_population() 
    best_of_run = None
    best_of_run_f = 0
    best_of_run_gen = 0
    fitnesses = [fitness(tree, dataset) for tree in population]

    # go evolution!
    for gen in range(GENERATIONS):        
        nextgen_population=[]
        for i in range(POP_SIZE):
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            parent1.crossover(parent2)
            parent1.mutation()
            nextgen_population.append(parent1)
        population=nextgen_population
        fitnesses = [fitness(tree, dataset) for tree in population]
        if max(fitnesses) > best_of_run_f:
            best_of_run_f = max(fitnesses)
            best_of_run_gen = gen
            best_of_run = deepcopy(population[fitnesses.index(max(fitnesses))])
#             print("________________________")
#             print("gen:", gen, ", best_of_run_f:", round(max(fitnesses),3), ", best_of_run:") 
#             best_of_run.print_tree()
        if best_of_run_f == 1: break   
    
#     print("\n\n_________________________________________________\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
#           " and has f=" + str(round(best_of_run_f,3)))
#     best_of_run.print_tree()
    return best_of_run
    

In [17]:
best_of_run = question4()
best_of_run.print_tree()

aBs
   mul
      sub
         cos
            3
         mul
            2
            x1
      x1
